In [ ]:
pip install peft

In [ ]:
pip install trl

In [ ]:
pip install py7zr

In [ ]:
pip install auto-gptq

In [ ]:
pip install bitsandbytes

In [ ]:
pip install accelerate

In [ ]:
pip install optimum

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset,Dataset

In [ ]:
df = load_dataset("samsum",split="train")

In [ ]:
df_data = df.to_pandas()

In [ ]:
df_data.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."


In [ ]:
data = df_data.sample(50)

In [ ]:
data.shape

(50, 3)

In [ ]:
data.reset_index(inplace=True,drop=True)

In [ ]:
data

In [ ]:
data['text'] = data[['dialogue' ,'summary']].apply(lambda x : "###Human :Summarize this following dialogue : " + x['dialogue'] + "\n###Assistant :" + x["summary"] ,axis = 1)

In [ ]:
data

In [ ]:
data['text'][49]

'###Human :Summarize this following dialogue : Victor: hey\r\nMason: hi \r\nVictor: would you like to go with me for a coffee? \r\nVictor: I need to talk \r\nMason: I have time tomorrow \r\nVictor: at 5?\r\nMason: sure \n###Assistant :Mason and Victor are meeting tomorrow at 5.'

In [ ]:
df = Dataset.from_pandas(data)

In [ ]:
df

Dataset({
    features: ['id', 'dialogue', 'summary', 'text'],
    num_rows: 50
})

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM, GPTQConfig, TrainingArguments,Trainer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
config = GPTQConfig(bits=4,disable_exllama =True,tokenizer=tokenizer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [ ]:
model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",config=config, device_map="auto")

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [ ]:
from peft import prepare_model_for_kbit_training

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from peft import get_peft_model

In [ ]:
peft_model = get_peft_model(model,lora_config)

In [ ]:
 training_args = TrainingArguments(
     output_dir="./results",
     per_device_train_batch_size=8,
     gradient_accumulation_steps=1,
     optim="paged_adamw_32bit",
     learning_rate=2e-4,
     lr_scheduler_type="cosine",
     save_strategy="epoch",
     save_total_limit=3,
     logging_steps=10,
     num_train_epochs=1,
     max_steps=100,
     fp16 = True,
 )

In [ ]:
from trl import SFTTrainer

In [ ]:
trainer  = SFTTrainer(
    model=peft_model,
    train_dataset=df,
    peft_config=lora_config,
    processing_class=tokenizer,  # Ensure this matches the library version
    args=training_args,
)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.336500
20,1.871600
30,1.694000
40,1.531500
50,1.399600
60,1.216100
70,1.093400
80,0.985800
90,0.948600
100,0.931700


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=100, training_loss=1.400874605178833, metrics={'train_runtime': 802.6495, 'train_samples_per_second': 0.997, 'train_steps_per_second': 0.125, 'total_flos': 204599574429696.0, 'train_loss': 1.400874605178833, 'epoch': 14.285714285714286})